In [1]:
import re
import json
import nltk
import numpy as np
import pandas as pd
import string
import os
import random
import codecs
import gensim
from collections import OrderedDict
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer 
from sklearn.model_selection import train_test_split

allbeers = pd.read_csv('../beers/beerlist.csv')
print('number of beers = '+(str(len(allbeers))))

number of beers = 4813


In [2]:
def pre_process(text):
    # lowercase
    text=text.lower()
    #remove tags
    text=re.sub("<!--?.*?-->","",text)
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    return text

allbeers['words'] = allbeers['words'].apply(lambda x:pre_process(x))

In [10]:
random.sample(allbeers['words'][0].split(), len(allbeers['words'][0]))

ValueError: Sample larger than population or is negative

In [ ]:
for i in range(len(allbeers)):
    allbeers['words'][i] = allbeers['words'][i].split()
#    try:
#        allbeers['words'][i] = random.sample(allbeers['words'][i], 2000)
#    except ValueError:
allbeers['words'][i] = random.sample(allbeers['words'][i], len(allbeers['words'][i]))
allbeers['words'][i] = ' '.join(allbeers['words'][i])

In [3]:
import pickle
# Function to remove stop words from sentences & lemmatize words.
def clean(beer):
    stop_free = " ".join([i for i in beer.lower().split() if i not in stop])
    normalized = " ".join(lemma.lemmatize(word,'v') for word in stop_free.split())
    x = normalized.split()
    y = [s for s in x if len(s) > 2]
    return y

In [4]:
#creating the dictionary
dictionary = gensim.corpora.Dictionary([" ".join(allbeers['words']).split()]) 
print('{} different terms in the corpus'.format(len(dictionary)))
#creating the bag of words object
bow_corpus = [dictionary.doc2bow(text.split()) for text in allbeers['words']]

168091 different terms in the corpus


In [5]:
tfidf_model = gensim.models.TfidfModel(bow_corpus) # creating the tf-idf model
tfidf_corpus = tfidf_model[bow_corpus]

In [10]:
total_topics = 14
lda_model_tfidf = gensim.models.LdaModel(corpus=tfidf_corpus, id2word=dictionary, num_topics=total_topics, passes=5)
lda_model_bow = gensim.models.LdaModel(corpus=bow_corpus, id2word=dictionary, num_topics=total_topics, passes=5)
ldatfidffile = open('lda_model_beers_14topics_tfidf.pkl','wb')
pickle.dump(lda_model_tfidf, ldatfidffile)
ldatfidffile.close()
ldabowfile = open('lda_model_beers_14topics_bow.pkl','wb')
pickle.dump(lda_model_bow, ldabowfile)
ldabowfile.close()

In [12]:
import pyLDAvis.gensim
data = pyLDAvis.gensim.prepare(lda_model_bow, bow_corpus, dictionary)
data

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker. The exit codes of the workers are {SIGKILL(-9)}

In [4]:
beer_words = allbeers['words']
#beers = open("docs_beers.pkl",'wb')
#pickle.dump(beer_words, beers)
 
# Use 3500 beers for training.
#beer_train, beer_test = train_test_split(beer_words, test_size=0.20)
beer_train = allbeers['words'][:3500]
beer_test = allbeers['words'][:3500].reset_index(drop=True)

# Cleaning all the train beers
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
#beer_clean = [clean(beer) for beer in beer_train]
beer_clean = [clean(beer) for beer in beer_words]

In [5]:
from gensim import corpora
# Creating term dictionary of corpus, where each unique term is assigned an index.
dictionary = corpora.Dictionary(beer_clean)

# Filter terms which occurs in less than 10 beers & more than 90% of the comments
dictionary.filter_extremes(no_below=10, no_above=0.9)
 
# List of few words which are removed from dictionary as they are content neutral
stoplist = set('bottle bad alcohol strong abv better good nothing beer beers also use make people know many call include part find become like mean often different usually take wikt come give well get since type list say change see refer actually iii aisne kinds pas ask would way something need things want every str'.split())
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)

In [6]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
beer_term_matrix = [dictionary.doc2bow(beer) for beer in beer_clean]

In [9]:
#from gensim.models import LdaMulticore as Lda
from gensim.models import LdaModel as Lda

# Creating the object for LDA model using gensim library & Training LDA model on the document term matrix.
#ldamodel = Lda(beer_term_matrix, workers=3, num_topics=14, id2word = dictionary, passes=10, iterations=200)
ldamodel = Lda(beer_term_matrix, num_topics=14, id2word = dictionary, passes=10, iterations=200)

# dump LDA model using cPickle for future use
ldafile = open('lda_model_beers_14topics_allbeers.pkl','wb')
pickle.dump(ldamodel,ldafile)
ldafile.close()
 
# Print all the 50 topics
for i,topic in enumerate(ldamodel.print_topics(num_topics=14, num_words=10)):
   words = topic[1].split("+")
   print(words,"\n")

['0.010*"corn" ', ' 0.008*"bottle" ', ' 0.007*"clear" ', ' 0.006*"water" ', ' 0.006*"grain" ', ' 0.006*"even" ', ' 0.006*"watery" ', ' 0.005*"lager" ', ' 0.005*"buy" ', ' 0.005*"time"'] 

['0.031*"sour" ', ' 0.023*"tart" ', ' 0.014*"fruit" ', ' 0.013*"bottle" ', ' 0.013*"funk" ', ' 0.011*"cherry" ', ' 0.010*"oak" ', ' 0.009*"tartness" ', ' 0.008*"red" ', ' 0.007*"wine"'] 

['0.034*"yeast" ', ' 0.028*"belgian" ', ' 0.019*"spice" ', ' 0.014*"bottle" ', ' 0.013*"banana" ', ' 0.011*"cleave" ', ' 0.011*"fruit" ', ' 0.010*"spicy" ', ' 0.009*"fruity" ', ' 0.008*"golden"'] 

['0.029*"caramel" ', ' 0.021*"amber" ', ' 0.013*"ale" ', ' 0.012*"malty" ', ' 0.010*"bottle" ', ' 0.010*"clear" ', ' 0.009*"copper" ', ' 0.009*"toast" ', ' 0.008*"rye" ', ' 0.008*"bitter"'] 

['0.054*"bourbon" ', ' 0.050*"vanilla" ', ' 0.036*"barrel" ', ' 0.036*"oak" ', ' 0.016*"age" ', ' 0.014*"maple" ', ' 0.013*"coconut" ', ' 0.013*"bottle" ', ' 0.012*"brown" ', ' 0.011*"whiskey"'] 

['0.014*"lager" ', ' 0.014*"crisp" ',

In [11]:
import pickle 
with open('lda_model_beers_14topics.pkl', 'rb') as fp:
    ldamodel = pickle.load(fp)

for i,topic in enumerate(ldamodel.print_topics(num_topics=14, num_words=10)):
   words = topic[1].split("+")
   print(words,"\n")

['0.037*"chocolate" ', ' 0.035*"dark" ', ' 0.033*"coffee" ', ' 0.029*"roast" ', ' 0.021*"black" ', ' 0.016*"brown" ', ' 0.015*"stout" ', ' 0.010*"porter" ', ' 0.010*"tan" ', ' 0.009*"creamy"'] 

['0.048*"spice" ', ' 0.045*"pumpkin" ', ' 0.030*"cinnamon" ', ' 0.021*"ginger" ', ' 0.014*"nutmeg" ', ' 0.010*"ale" ', ' 0.009*"pie" ', ' 0.007*"vanilla" ', ' 0.007*"brown" ', ' 0.007*"dark"'] 

['0.027*"dark" ', ' 0.024*"smoke" ', ' 0.016*"rye" ', ' 0.016*"brown" ', ' 0.014*"fruit" ', ' 0.012*"caramel" ', ' 0.008*"sugar" ', ' 0.005*"spice" ', ' 0.005*"tan" ', ' 0.005*"yeast"'] 

['0.027*"bourbon" ', ' 0.018*"vanilla" ', ' 0.015*"barrel" ', ' 0.015*"dark" ', ' 0.013*"oak" ', ' 0.012*"caramel" ', ' 0.011*"age" ', ' 0.010*"brown" ', ' 0.010*"fruit" ', ' 0.008*"pepper"'] 

['0.011*"tea" ', ' 0.011*"spruce" ', ' 0.010*"dark" ', ' 0.009*"black" ', ' 0.008*"guinness" ', ' 0.008*"tequila" ', ' 0.006*"roast" ', ' 0.006*"agave" ', ' 0.006*"ale" ', ' 0.006*"interest"'] 

['0.024*"honey" ', ' 0.014*"lager

In [7]:
import pickle
# Function to remove stop words from sentences & lemmatize words.
def clean(beer):
    stop_free = " ".join([i for i in beer.lower().split() if i not in stop])
    normalized = " ".join(lemma.lemmatize(word,'v') for word in stop_free.split())
    x = normalized.split()
    y = [s for s in x if len(s) > 2]
    return y

In [19]:
def cluster_similar_documents(corpus, dirname):
    clean_beers = [clean(beer) for beer in corpus]
    test_term = [ldamodel.id2word.doc2bow(beer) for beer in clean_beers]
    beer_topics = ldamodel.get_document_topics(test_term, minimum_probability=0.20)
    for k,topics in enumerate(beer_topics):
        if topics:
            topics.sort(key = itemgetter(1), reverse=True)
            dir_name = dirname + "/" + str(topics[0][0])
            file_name = dir_name + "/" + str(k) + ".txt"
            if not os.path.exists(dir_name):
                os.makedirs(dir_name)
            fp = open(file_name,"w")
            fp.write(beers_test[k] + "\n\n" + str(topics[0][1]) )
            fp.close()
        else:
            if not os.path.exists(dirname + "/unknown"):
                os.makedirs(dirname + "/unknown")
            file_name = dirname + "/unknown/" + str(k) + ".txt"
            fp = open(file_name,"w")
            fp.write(beers_test[k])

def get_related_documents(term, top, corpus):
    clean_docs = [clean(doc) for doc in corpus]
    related_docid = []
    test_term = [ldamodel.id2word.doc2bow(doc) for doc in clean_docs]
    doc_topics = ldamodel.get_document_topics(test_term, minimum_probability=0.30)
    term_topics =  ldamodel.get_term_topics(term, minimum_probability=0.01)
    for k,topics in enumerate(doc_topics):
        if topics:
            topics.sort(key = itemgetter(1), reverse=True)
            try:
                if topics[0][0] == term_topics[0][0]:
                    related_docid.append((k,topics[0][1]))
            except IndexError:
                return print("Too specific or no match! Try again.")
    beerids = []
    related_docid.sort(key = itemgetter(1), reverse=True)
    for j,doc_id in enumerate(related_docid):
        print(doc_id[1],"\n\n",docs_test[doc_id[0]])
        if j == (top-1):
            break
        beerids.append(doc_id[0])
    if len(beerids) > 0:
        return beerids #doc_id[0]
    else:
        return "Too specific or no match! Try again"

In [9]:
docs_fp = open("docs_beers.pkl", 'rb')
docs_all = pickle.load(docs_fp)
docs_test = docs_all#[3500:].reset_index(drop=True)

In [ ]:
a = get_related_documents("stout", 10 , docs_test)

In [28]:
allbeers['name'][0]

'The Big Smoke'

In [12]:
import pandas as pd
allbeers = pd.read_csv('../beers/beerlist.csv')

In [16]:
#beers = pd.read_csv('../beers/beerlist.csv')
#beers['name'][a]
for i in a:
    print(allbeers['name'][i])

Poop Your Pants Chocolate Bock
Lobo Negro
Ellie's Brown Ale
Rugged Trail Nut Brown Ale
Mad Tom's Robust Porter
Boffo Brown Ale
Jupiter (The Bringer Of Jollity)
Samuel Adams Hazel Brown
Get Up Offa That Brown


In [2]:
import pickle
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from operator import itemgetter
import os
 
# initialize WordNetLemmatizer and get the list of english stop words
stop = set(stopwords.words('english'))
lemma = WordNetLemmatizer()
 
# Load trained LDA model
lda_fp = open("lda_model_beers_14topics_allbeers.pkl", 'rb')
ldamodel = pickle.load(lda_fp)
 
# Load the beers corpus to choose 500 lines for test purpose
beers_fp = open("docs_beers.pkl", 'rb')
#beers_all = pickle.load(docs_fp)
#_test = docs_all[3500:].reset_index(drop=True)

In [10]:
beers_all = pickle.load(beers_fp)
beer_test = beers_all[3500:].reset_index(drop=True)

In [28]:
# Get 'top' related documents given a word(term)
a = get_related_documents("chocolate", 10 , beer_test)
# performs document clustering given a set of documents
#cluster_similar_documents(docs_test,"root")

0.99922216


In [10]:
import pickle 
with open('lda_model_beers_14topics.pkl', 'rb') as fp:
    lda_dict = pickle.load(fp)

In [16]:
lda_dict.show_topic(topicid=8)

[('wheat', 0.020052893),
 ('yeast', 0.019697063),
 ('belgian', 0.015831603),
 ('banana', 0.0151973935),
 ('spice', 0.014571979),
 ('orange', 0.010813807),
 ('cleave', 0.009907254),
 ('hazy', 0.008648241),
 ('fruit', 0.008244533),
 ('citrus', 0.007832508)]

In [4]:
import pandas as pd
allbeers = pd.read_csv('../beers/beerlist.csv')

In [30]:
allbeers = pd.read_csv('../beers/beerlist.csv')
beer_train = allbeers['words'][:3500]
beer_test = allbeers['words'][:3500].reset_index(drop=True)

In [66]:
beer_test[0]
words = [re.sub(r'[^\w\s]', '', word) for word in beer_test[0]]

In [89]:
words = pre_process(beer_test[0])
words = clean(words)

In [87]:
beer_clean = [pre_process(clean(beer_test)) for beer in beer_test]

AttributeError: 'Series' object has no attribute 'lower'

In [79]:
# select and article at random from test_df
random_article_index = np.random.randint(len(words))
print(random_article_index)

new_bow = dictionary.doc2bow(words[random_article_index])

4515


TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [92]:
import gensim 
gensim.matutils.cossim()

<module 'gensim.matutils' from '/home/steven/anaconda2/lib/python3.7/site-packages/gensim/matutils.py'>

In [43]:
beer_test.iloc[random_article_index]

'['

In [5]:
words0 = clean(pre_process(beer_test[0]))
words1 = clean(pre_process(beer_test[1]))
words2 = clean(pre_process(beer_test[2]))
words3 = clean(pre_process(beer_test[3]))

d0 = dictionary.doc2bow(words0)
d1 = dictionary.doc2bow(words1)
d2 = dictionary.doc2bow(words2)
d3 = dictionary.doc2bow(words3)

NameError: name 'clean' is not defined

In [4]:
lda_dict(allbeers['words'][0])

TypeError: 'LdaMulticore' object is not callable